# Random Forest

In [1]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import os

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

In [3]:
merge_features = '"{}"'.format(os.path.join(current_folder, '..', 'Features', 'Merge features.ipynb'))
calcular_auc = '"{}"'.format(os.path.join(current_folder, '..', 'Calcular AUC.ipynb'))
set_de_entrenamiento_testing_y_prediccion = '"{}"'.format(os.path.join(
    current_folder,
    '..',
    'Set de entrenamiento, testing y predicción.ipynb'
))

In [4]:
pd.options.mode.chained_assignment = None
%run $merge_features

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

In [5]:
assert(df_features.shape[0] == df['person'].unique().shape[0])

Cargo los sets de entrenamiento, testing y predicción.

In [6]:
%run $set_de_entrenamiento_testing_y_prediccion

## Entrenamiento rápido

In [7]:
training = labels_training.merge(df_features, how='inner', on='person')

In [8]:
columns = list(training.columns)
columns.remove('label')

In [9]:
X = training.loc[:, columns]
y = training['label']

Hiperparámetros:

In [10]:
param = {'n_estimators':100, 'max_depth':10}

In [11]:
regr = RandomForestRegressor(**param)
regr.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

Predigo:

In [12]:
labels_test = labels_test.merge(df_features, how='inner', on='person')

In [13]:
labels_test['label_predicted'] = regr.predict(labels_test.loc[:, columns])

In [14]:
%run $calcular_auc

auc score: 0.8471460675302439


In [15]:
feature_importance = pd.DataFrame(data={'columna':columns, 'importancia':regr.feature_importances_}).set_index('columna')
feature_importance.sort_values('importancia', ascending=False)

,importancia
columna,
checkout,0.156442
days until 31-05 mean,0.081028
days until 31-05 std,0.053098
event_count,0.035784
Cant visitas con Computadoras,0.030382
viewed product,0.027198
ad campaign hit,0.025985
Cant visitas con smartphone,0.023247
generic listing,0.022664
